In [1]:
# import modules + data
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objects as go



In [ ]:
# graphing functions to make below less difficult
def bubble(x, y, color, size, labels):
    data = [go.Scatter (
        x = x,
        y = y,
        text=labels,
        mode = 'markers',
        marker = dict(
            color=color,
            size=size,
            sizemode='area',
            sizeref=2.*max(size)/(45.**2),
            sizemin=4))]
    fig = go.Figure(data)
    fig.show()

def bar(x, y, colors):
    trace1 = go.Bar(
        x=x,
        y=y,
        marker_color=colors)
    fig = go.Figure([trace1])
    fig.show()
    
def barStack(labels, y1, y2):
    trace1 = go.Bar(
        x=labels,
        y=y1)
    trace2 = go.Bar(
        x=labels,
        y=y2)
    fig = go.Figure([trace1, trace2])
    fig.update_layout(barmode='stack')
    fig.show()    


In [ ]:
# check how many parties there are and what they are. and if any new footnotes are up
pop_sorted_data['Party'].unique()

# set up what party gets what colors and transform party column in data into a corresponding list of colors
colordict = {'Republican': 'red', 'Democratic':'blue', 'Independent':'gray', 'Vacant':'purple'}
colors = [colordict[k] for k in list(pop_sorted_data['Party'])]
# NOTE - make this robust re footnotes


In [ ]:
# @TODO: Make this unnecessary? Ideally would work for any # of parties with any names, just pull from one big dataset?
# that might be easier if plotted in pandas instead of plotly tbf

# Get lists of various variables so they can be easily plugged into plotly
R_data = sen_with_pop[sen_with_pop['Party'] == 'Republican']
R_data.head()
R_pops = list(R_data['Population'])
R_sen_list = list(R_data['Senator'])
R_states = list(R_data['State'])

D_data = sen_with_pop[sen_with_pop['Party'] != 'Republican']
D_data.head()
D_pops = list(D_data['Population']) # includes independents
D_sen_list = list(D_data['Senator'])
D_states = list(D_data['State'])

I_data = sen_with_pop[sen_with_pop['Party'] == 'Independent[a]']
I_pops = list(I_data['Population'])

In [ ]:
# this builds the coordinates for a senate chamber setup
# @TODO: Republicans are currently on the LEFT

R_coords = []
for i in range(10):
    for j in range(5):
        R_coords.append([i, j])

D_coords = []
for i in range(10, 20):
    for j in range(5):
        D_coords.append([i, j])

In [ ]:
# one-to-one list of coloring for any list going [republicans, democrats]
colors = ['red'] * 50 + ['blue'] * 50

# see this is just silly
full_sen_list = R_sen_list + D_sen_list
state_list = R_states + D_states

In [ ]:
# bubble plot of population of each state, with mouseover labels for each senator
x_vals = [a[0] for a in (R_coords + D_coords)]
y_vals = [a[1] for a in (R_coords + D_coords)]
pops = [a for a in (R_pops + D_pops)]

bubble(x_vals, y_vals, colors, pops, full_sen_list)

In [ ]:
# summary stats for each party
all_Rs = sum(R_pops)
all_Ds = sum(D_pops)
just_Is = sum(I_pops)

In [ ]:
# SEE THIS IS THE WAY TO DO IT
# admittedly sorting it and relying on that sort might be a little weak? but GOOD

# anyway this is the data sorted by state population, lowest to highest
pop_sorted_data = sen_with_pop.sort_values(by='Population').reset_index(drop=True)
pop_sorted_data.head()

In [ ]:
# Should this not be with the barstack function


# Stacked bar chart (for 2 senators per state)

# since data is sorted by population, senators from the same state must be adjacent
# so taking every other one gives one from each state
stack_1 = sen_with_pop[sen_with_pop.index % 2 == 0]
stack_2 = sen_with_pop[sen_with_pop.index % 2 == 1]

# colors stuff, as above
colors1 = [colordict[k] for k in list(stack_1['Party'])]
colors2 = [colordict[k] for k in list(stack_2['Party'])]


# bottom half of bar chart
trace1 = go.Bar(
    x=list(stack_1['State']),
    y=list(stack_1['Population']/2), # divide state pop in two so two senators makes the right pop come out
    # But don't want hovering to SHOW this
    marker_color = colors1,
    text=list(stack_1['State'])) # hovering shows state

# top half, same thing
trace2 = go.Bar(
    x=list(stack_1['State']),
    y=list(stack_2['Population']/2),
    marker_color=colors2,
    hovertext=list(stack_2['State'])) # maybe do abbreviations? 

# make chart!
fig = go.Figure([trace1])
fig.add_trace(trace2)

# add a line between 9 and 10 to show the halfway point
# @TODO: make robust against different distributions of population
fig.add_vline(x=9.5, line_dash='dot')

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()    


In [ ]:
# very simple graph showing how many people democrats vs republicans represent
mylabels = ['Republican', 'Democrat']

bar(mylabels, [all_Rs/2, all_Ds/2], ['red', 'blue']) #dividing by 2 because 2 senators

In [ ]:
sort_new = sen_with_pop.sort_values(by='Population')
sort_new.head()

Rns = sort_new[sort_new['Party'] == 'Republican']
Dns = sort_new[sort_new['Party'] != 'Republican']

In [ ]:
# JUST MAKE A POLAR CHART FFS



fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}]*2]*1, horizontal_spacing=0)

rep_coords = [(r, a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]
dem_coords = [(r, 180 - a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]

r_vals = [a[0] for a in dem_coords] + [a[0] for a in rep_coords]
theta_vals = [a[1] for a in dem_coords] + [a[1] for a in rep_coords]



fig.add_trace(go.Scatterpolar(
        r = [a[0] for a in dem_coords],
        theta = [a[1] for a in dem_coords],
        text=list(Dns['Senator']),
        mode = 'markers',
        marker = dict(
            color='blue',
            size=list(Dns['Population']),
            sizemode='area',
            sizeref=2.*40000000/(50.**2),
            sizemin=4)

    ), 1, 1)

fig.add_trace(go.Scatterpolar(
        r = [a[0] for a in rep_coords],
        theta = [a[1] for a in rep_coords],
        text = list(Rns['Senator']),
        mode = 'markers',
        marker = dict(
            color='red',
            size=list(Rns['Population']),
            sizemode='area',
            sizeref=2.*40000000/(50.**2),
            sizemin=4)
    ), 1, 2)



fig.update_layout(
    title = "US Senate",
    font_size = 15,
    showlegend = False,
    polar = dict(
        sector = [90,180],
        bgcolor = "white",
        angularaxis = dict(showline=False,showticklabels=False, ticks=''),
        radialaxis = dict(showline=False, showticklabels=False, ticks='')
    ),
    polar2 = dict(
        sector = [0, 90],
        bgcolor = "white",
        angularaxis = dict(showline=False,showticklabels=False, ticks=''),
        radialaxis = dict(showline = False,showticklabels=False, ticks='')
    ),
    paper_bgcolor = "white"
)



fig.show()